<a href="https://colab.research.google.com/github/turkfuat/KiTS19-Hybird-V-Net-Model/blob/main/Train_YoloV3_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# To Check GPU
!nvidia-smi

Mon Mar 13 18:54:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# For mounting Drive to Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 Çizelge.gsheet
'Colab Notebooks'
 custom_spot_model
'darknet.zip adlı dosyanın kopyası'
 FİNAL.zip
 forms.html
 images
'My Drive'
'sagliksiz_ (1).jpg'
'SALGIN HASTALIKLARINDA MESAFE DEDEKTÖRÜ SEFA KARTAL 190905027.zip'
 sketch_dec13a
 spot_model.ipynb
 spot_weights
'Staj Defteri_EEM Mart 2020 (1).docx'
'Staj Defteri_EEM Mart 2020 (1).pdf'
'Staj Defteri_EEM Mart 2020.docx'
 STAJJ-RİYAT-1.docx
 STAJJ-RİYAT-1.gdoc
 VID-20221117-WA0002.mp4
 yolov3
 yolov3_1
 yolov3_2


**1) Clone the Darknet**



In [ ]:
# Clone the Darknet to the Colab
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15514, done.
remote: Total 15514 (delta 0), reused 0 (delta 0), pack-reused 15514
Receiving objects: 100% (15514/15514), 14.17 MiB | 6.94 MiB/s, done.
Resolving deltas: 100% (10412/10412), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [ ]:
# Configuration file (cfg file)
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Setting for single object

!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3/"

mkdir: cannot create directory ‘/mydrive/yolov3/’: File exists


In [ ]:
# Jet is a class name
!echo "Teeth" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3/' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-03-13 18:57:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  16.0MB/s    in 15s     

2023-03-13 18:58:04 (10.2 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
# Extracting custom object dataset
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/sagliksiz_(1).jpg  
  inflating: data/obj/sagliksiz_(1).txt  
  inflating: data/obj/sagliksiz_(10).txt  
  inflating: data/obj/sagliksiz_(101).txt  
  inflating: data/obj/sagliksiz_(102).txt  
  inflating: data/obj/sagliksiz_(103).txt  
  inflating: data/obj/sagliksiz_(104).txt  
  inflating: data/obj/sagliksiz_(105).txt  
  inflating: data/obj/sagliksiz_(108).txt  
  inflating: data/obj/sagliksiz_(109).txt  
  inflating: data/obj/sagliksiz_(11).txt  
  inflating: data/obj/sagliksiz_(110).txt  
  inflating: data/obj/sagliksiz_(111).txt  
  inflating: data/obj/sagliksiz_(112).txt  
  inflating: data/obj/sagliksiz_(113).txt  
  inflating: data/obj/sagliksiz_(114).txt  
  inflating: data/obj/sagliksiz_(116).txt  
  inflating: data/obj/sagliksiz_(119).txt  
  inflating: data/obj/sagliksiz_(120).txt  
  inflating: data/obj/sagliksiz_(121).txt  
  inflating: data/obj/sagliksiz_(122).txt  
  inflating: data/obj/sagliksiz_(126).txt  
 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.724292 0.597730 0.084906 0.150354
['0', '0.724292', '0.597730', '0.084906', '0.150354']
0 data/obj/sagliksiz_(134).txt
0 0.724292 0.597730 0.084906 0.150354
0 0.737264 0.377211 0.075472 0.185436
['0', '0.737264', '0.377211', '0.075472', '0.185436']
1 data/obj/sagliksiz_(265).txt
0 0.737264 0.377211 0.075472 0.185436
0 0.742571 0.472435 0.069575 0.150354

['0', '0.742571', '0.472435', '0.069575', '0.150354']
2 data/obj/sagliksiz_(24).txt
0 0.742571 0.472435 0.069575 0.150354
0 0.823349 0.644089 0.075472 0.142836

['0', '0.823349', '0.644089', '0.075472', '0.142836']
2 data/obj/sagliksiz_(24).txt
0 0.823349 0.644089 0.075472 0.142836
0 0.383491 0.466170 0.047170 0.117777
['0', '0.383491', '0.466170', '0.047170', '0.117777']
2 data/obj/sagliksiz_(24).txt
0 0.383491 0.466170 0.047170 0.117777
0 0.809788 0.396005 0.074292 0.162883

['0', '0.809788', '0.396005', '0.074292', '0.162883']
3 data/obj/sagliksiz_(76).txt
0 0.809788 0.396005 0.074292 0.162883
0 0.871698 0.377211 0.049528 0.1704

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/sagliksiz_(428).jpg', 'data/obj/sagliksiz_(426).jpg', 'data/obj/sagliksiz_(448).jpg', 'data/obj/sagliksiz_(141).jpg', 'data/obj/sagliksiz_(111).jpg', 'data/obj/sagliksiz_(16).jpg', 'data/obj/sagliksiz_(41).jpg', 'data/obj/sagliksiz_(31).jpg', 'data/obj/sagliksiz_(260).jpg', 'data/obj/sagliksiz_(46).jpg', 'data/obj/sagliksiz_(419).jpg', 'data/obj/sagliksiz_(52).jpg', 'data/obj/sagliksiz_(367).jpg', 'data/obj/sagliksiz_(381).jpg', 'data/obj/sagliksiz_(424).jpg', 'data/obj/sagliksiz_(38).jpg', 'data/obj/sagliksiz_(411).jpg', 'data/obj/sagliksiz_(56).jpg', 'data/obj/sagliksiz_(402).jpg', 'data/obj/sagliksiz_(373).jpg', 'data/obj/sagliksiz_(150).jpg', 'data/obj/sagliksiz_(199).jpg', 'data/obj/sagliksiz_(400).jpg', 'data/obj/sagliksiz_(272).jpg', 'data/obj/sagliksiz_(224).jpg', 'data/obj/sagliksiz_(271).jpg', 'data/obj/sagliksiz_(112).jpg', 'data/obj/sagliksiz_(45).jpg', 'data/obj/sagliksiz_(220).jpg', 'data/obj/sagliksiz_(212).jpg', 'data/obj/sagliksiz_(154).jpg', 'data/obj/sagli

In [ ]:
#Create training.txt file, required for YOLOv3 training
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
# Calculates anchors for the custom dataset
!./darknet detector calc_anchors data/obj.data -num_of_clusters 9 -width 608 -height 608

 CUDA-version: 11080 (12000), cuDNN: 8.7.0, GPU count: 1  
 OpenCV version: 4.2.0

 num_of_clusters = 9, width = 608, height = 608 
 read labels from 345 images 
 loaded 	 image: 345 	 box: 869
 all loaded. 

 calculating k-means++ ...

 iterations = 27 


counters_per_class = 869

 avg IoU = 88.10 % 

Saving anchors to the file: anchors.txt 
anchors =  27, 55,  39, 55,  34, 66,  30, 83,  38, 76,  45, 66,  38, 97,  49, 81,  51,102
^C


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Görüntülenen çıkış son 5000 satıra kısaltıldı.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.809404), count: 1, class_loss = 0.235239, iou_loss = 0.019534, total_loss = 0.254773 
 total_bbox = 233515, rewritten_bbox = 0.008993 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.853301), count: 8, class_loss = 0.480780, iou_loss = 0.125259, total_loss = 0.606038 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.799549), count: 5, class_loss = 0.784296, iou_loss = 0.177991, total_loss = 0.962287 
 total_bbox = 233528, rewritten_bbox = 0.008992 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000026, iou_loss = 0.000000, total_loss = 0.000026 
v3 (mse l

In [ ]:
# Training on last saved weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg '/content/gdrive/My Drive/yolov3/yolov3_training_last.weights' -dont_show

Görüntülenen çıkış son 5000 satıra kısaltıldı.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000100, iou_loss = 0.000000, total_loss = 0.000100 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.785641), count: 1, class_loss = 0.022351, iou_loss = 0.020127, total_loss = 0.042479 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.771078), count: 5, class_loss = 0.097219, iou_loss = 0.377738, total_loss = 0.474957 
 total_bbox = 64777, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.837692), count: 6, class_loss = 0.049458, iou_loss = 0.133557, total_loss = 0.183015 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Re

In [ ]:
# Testing on an image
!./darknet detect cfg/yolov3_testing.cfg '/content/gdrive/My Drive/yolov3/yolov3_training_final.weights' data/sagliksiz_(1).jpg

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `./darknet detect cfg/yolov3_testing.cfg '/content/gdrive/My Drive/yolov3/yolov3_training_final.weights' data/sagliksiz_(1).jpg'


In [ ]:
# Mean Average Precision (mAP) Calculation
!./darknet detector map data/obj.data cfg/yolov3_training.cfg '/content/gdrive/My Drive/yolov3/yolov3_trained.weights'

 CUDA-version: 11060 (11060), cuDNN: 8.4.0, GPU count: 1  
 OpenCV version: 4.2.0
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 16, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5, 